In [11]:
%%html
<style>
.cell-output-ipywidget-background { 
  background-color: transparent !important; 
}
</style>

In [12]:
import math, random
from time import sleep
from ipycanvas import Canvas, hold_canvas

In [13]:
canvas = Canvas(width=1000, height=1000)
canvas.layout.width = "50%"
canvas.layout.height = "auto"
width = canvas.width
height = canvas.height

In [14]:
def init():
  global width, height, canvas
  global sz, gap, delay, decay, pcnt, cw, ch, done, state, newstate, gray
  sz = 10
  gap = 1
  decay = 0.99
  pcnt = 0.35
  cw = math.floor((width - gap) / (sz + gap))
  ch = math.floor((height - gap) / (sz + gap))
  state = []
  newstate = []
  gray = []
  #
  canvas.fill_style = "rgb(20,20,20)"
  width = cw * (sz + gap) + gap
  height = ch * (sz + gap) + gap
  canvas.fill_rect(0, 0, width, height)
  #
  for i in range(ch):
    state.append([ 0 ] * cw)
    newstate.append([ 0 ] * cw)
    gray.append([ 0 ] * cw)
  for i in range(ch):
    for j in range(cw):
      state[i][j] = 1 if random.random() > pcnt else 0
      gray[i][j] = 0 if state[i][j] else 255

In [15]:
def drawCell(i, j, color):
  global gap, sz
  canvas.fill_style = color
  canvas.fill_rect(gap + i * (sz + gap), gap + j * (sz + gap), sz, sz)

In [16]:
def drawCells():
  global state, gray, decay
  with hold_canvas():
    batch_x = {}
    batch_y = {}
    for i in range(ch):
      for j in range(cw):
        val = 0 if state[i][j] else 255
        if val == 0:
          gray[i][j] = val
        else:
          val = gray[i][j] = math.ceil(
            decay * gray[i][j] + (1 - decay) * val)
          val = math.floor(256 + (val - 256) / 2)
        val = 255 - val
        # color = "rgb({},{},{})".format(val, val, val)
        # drawCell(i, j, color)
        xs = batch_x.setdefault(val, [])
        ys = batch_y.setdefault(val, [])
        xs.append(gap + i * (sz + gap))
        ys.append(gap + j * (sz + gap))
    for val in batch_x:
      xs = batch_x[val]
      ys = batch_y[val]
      color = "rgb({},{},{})".format(val, val, val)
      canvas.fill_style = color
      canvas.fill_rects(xs, ys, sz)

In [17]:
def updateState():
  global state, newstate
  for i in range(ch):
    for j in range(cw):
      sum = 0
      for k in range(-1, 2):
        for l in range(-1, 2):
          if k == 0 and l == 0: continue
          ni = (i + k + ch) % ch
          nj = (j + l + cw) % cw
          sum += state[ni][nj]
      if sum < 2 or sum > 3:
        newstate[i][j] = 0
      elif sum == 3:
        newstate[i][j] = 1
      else:
        newstate[i][j] = state[i][j]
  temp = state
  state = newstate
  newstate = temp

In [18]:
def loop():
  updateState()
  drawCells()
  canvas.sleep(10)
  sleep(0.010)

def run(doinit, n):
  if doinit: init()
  for i in range(n): loop()

run(1, 0)
canvas

Canvas(height=1000, layout=Layout(height='auto', width='50%'), width=1000)

In [19]:
run(0, 100)